In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
%%time
statements_df = pd.read_csv('statements.csv')
statements_df

Wall time: 38.9 s


,source_item_id,edge_property_id,target_item_id
0,1,31,36906466
1,1,279,3695190
2,1,398,497745
3,1,398,1133705
4,1,398,1139177
...,...,...,...
141206848,77257484,59,9286
141206849,77257491,31,318
141206850,77257491,59,9286
141206851,77257493,31,318


In [ ]:
page_df = pd.read_csv('page.csv')
page_dict = pd.Series(page_df.page_id.values,index=page_df.item_id).to_dict()
del(page_df)
page_dict[6199]

In [9]:
%%time
statements_df_page = statements_df.copy()
statements_df_page['source_page_id'] = [page_dict.get(i) for i in statements_df_page.source_item_id]
statements_df_page['target_page_id'] = [page_dict.get(i) for i in statements_df_page.target_item_id]
statements_df_page

Wall time: 2min 44s


,source_item_id,edge_property_id,target_item_id,source_page_id,target_page_id
0,1,31,36906466,31880.0,NaN
1,1,279,3695190,31880.0,50723558.0
2,1,398,497745,31880.0,6207679.0
3,1,398,1133705,31880.0,17365007.0
4,1,398,1139177,31880.0,435544.0
...,...,...,...,...,...
141206848,77257484,59,9286,NaN,152710.0
141206849,77257491,31,318,NaN,12558.0
141206850,77257491,59,9286,NaN,152710.0
141206851,77257493,31,318,NaN,12558.0


In [10]:
statements_df_page.drop(columns=['source_item_id', 'target_item_id'], inplace=True)
statements_df_page = statements_df_page.dropna()
statements_df_page = statements_df_page[['source_page_id', 'edge_property_id', 'target_page_id']]
statements_df_page

,source_page_id,edge_property_id,target_page_id
1,31880.0,279,50723558.0
2,31880.0,398,6207679.0
3,31880.0,398,17365007.0
4,31880.0,398,435544.0
5,31880.0,398,33196876.0
...,...,...,...
140846125,2368655.0,59,105200.0
140846126,2368655.0,361,962148.0
140853808,62057360.0,31,528282.0
140857402,62276323.0,31,682482.0


In [3]:
#Choose which type of graph you want
#If you want item use 'statements_df'
#If you want page use 'statements_df_page', this is much smaller than item

statements_np = statements_df.to_numpy(dtype=int)
del(statements_df)
statements_np

array([[       1,       31, 36906466],
       [       1,      279,  3695190],
       [       1,      398,   497745],
       ...,
       [77257491,       59,     9286],
       [77257493,       31,      318],
       [77257493,       59,     8913]])

In [ ]:
#RUN THIS IS YOU WANT A BI-DIRECTIONAL GRAPH
#statements_np = np.concatenate((statements_np, statements_np[:, [2, 1, 0]]), axis=0)

In [4]:
%%time
statements_np = statements_np[np.argsort(statements_np[:, 0])]
statements_np

Wall time: 7.35 s


array([[       1,       31, 36906466],
       [       1,     1343,   602358],
       [       1,     1419,  5457948],
       ...,
       [77257491,       59,     9286],
       [77257493,       31,      318],
       [77257493,       59,     8913]])

In [5]:
u, indices = np.unique(statements_np[:,0], return_index=True)

In [6]:
indices1 = np.append(indices, len(statements_np))
indices1[-10:]

array([141206835, 141206837, 141206839, 141206841, 141206843, 141206845,
       141206847, 141206849, 141206851, 141206853], dtype=int64)

In [7]:
%%time
statement_dict = {j:statements_np[indices1[i]:indices1[i+1], :0:-1] for i, j in enumerate(u)}
del(statements_np)
print(len(statement_dict.items()))
statement_dict

51106679
Wall time: 1min 18s


{1: array([[36906466,       31],
        [  602358,     1343],
        [ 5457948,     1419],
        [22903368,     1424],
        [   11412,     1552],
        [  221392,     1889],
        [ 2708714,     1889],
        [  136407,     2184],
        [     338,     2579],
        [    6999,     2670],
        [   18343,     2670],
        [   79925,     2670],
        [ 2504088,     2670],
        [ 6005984,     2670],
        [28732711,     2670],
        [ 5460604,     5008],
        [ 5551050,      910],
        [     323,      828],
        [  185674,     2670],
        [ 1208634,      793],
        [ 3491753,      793],
        [ 3695190,      279],
        [  497745,      398],
        [ 1139177,      398],
        [ 7439451,      398],
        [15241043,      398],
        [17863945,      398],
        [23054687,      398],
        [ 1133705,      398],
        [  133327,      527],
        [  221392,      527],
        [     323,      793],
        [  273508,      793],
       

In [ ]:
%%time
#takes ~1 min for page
#takes ~3 min for item
with open("graph_item_uni.p", 'wb') as f:
    pickle.dump(statement_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
%%time
with open('graph_uni.p', 'rb') as fp:
    aaa = pickle.load(fp)
aaa

Wall time: 12.2 ms


{1: array([[36906466,       31],
        [  602358,     1343],
        [ 5457948,     1419],
        [22903368,     1424],
        [   11412,     1552],
        [  221392,     1889],
        [ 2708714,     1889],
        [  136407,     2184],
        [     338,     2579],
        [    6999,     2670],
        [   18343,     2670],
        [   79925,     2670],
        [ 2504088,     2670],
        [ 6005984,     2670],
        [28732711,     2670],
        [ 5460604,     5008],
        [ 5551050,      910],
        [     323,      828],
        [  185674,     2670],
        [ 1208634,      793],
        [ 3491753,      793],
        [ 3695190,      279],
        [  497745,      398],
        [ 1139177,      398],
        [ 7439451,      398],
        [15241043,      398],
        [17863945,      398],
        [23054687,      398],
        [ 1133705,      398],
        [  133327,      527],
        [  221392,      527],
        [     323,      793],
        [  273508,      793],
       

In [6]:
%%time
statement_dict = {j:target_edge[indices1[i]:indices1[i+1]] for i, j in enumerate(u)}
print(len(statement_dict.items()))
del(u, indices, indices1, target_edge)
statement_dict[1]

51106679
Wall time: 2min 20s


((36906466, 31),
 (3695190, 279),
 (497745, 398),
 (1133705, 398),
 (1139177, 398),
 (7439451, 398),
 (15241043, 398),
 (17863945, 398),
 (23054687, 398),
 (2051667, 461),
 (133327, 527),
 (221392, 527),
 (323, 793),
 (273508, 793),
 (837317, 793),
 (1079806, 793),
 (1079826, 793),
 (1208634, 793),
 (3491753, 793),
 (323, 828),
 (5551050, 910),
 (602358, 1343),
 (5457948, 1419),
 (22903368, 1424),
 (11412, 1552),
 (221392, 1889),
 (2708714, 1889),
 (136407, 2184),
 (338, 2579),
 (6999, 2670),
 (18343, 2670),
 (79925, 2670),
 (185674, 2670),
 (2504088, 2670),
 (6005984, 2670),
 (28732711, 2670),
 (5460604, 5008))